In [1]:
## Install necessary packages
#pip install geopandas 
#pip install folium
# To install the package


import pandas as pd
import numpy as np
from pandas import DataFrame

import geopandas
import matplotlib.pyplot as plt

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print ("done importing packages")


done importing packages


In [8]:
pip install geopandas 

     |████████████████████████████████| 962 kB 14.9 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 32.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 36.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install folium

     |████████████████████████████████| 94 kB 3.2 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Read the data
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
# Get table                                                                                                           
df = dfs[0]


In [3]:
#check the data looks correct
df.shape

(180, 3)

In [4]:
#check the data looks correct
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
#creating a copy of the data so we dont mess with the original 
df_copy= df


# Creating a cleaned DF copy that will not include Bouroughs that are Not assigned

In [6]:
df_cleaned = df_copy[df_copy['Borough'] != "Not assigned"]
df_cleaned.head(20)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


# Using a CSV to get coordinates of the postal codes in Toronto since Geocoder did not work

In [7]:
geo_data = "https://cocl.us/Geospatial_data"
geo_df= pd.read_csv(geo_data)
print(geo_df.shape)
geo_df.head()


(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
merged_df= pd.merge(df_cleaned, geo_df, on= 'Postal Code')
merged_df.shape

(103, 5)

In [9]:
merged_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [10]:
# combining Latitude and longitude
merged_df['Longitude']= merged_df['Longitude'].astype(str)
merged_df['Latitude']= merged_df['Latitude'].astype(str)
merged_df['Coordinates'] = merged_df['Latitude'].str.cat(merged_df['Longitude'],sep=", ")

In [11]:
merged_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Coordinates
0,M3A,North York,Parkwoods,43.7532586,-79.3296565,"43.7532586, -79.3296565"
1,M4A,North York,Victoria Village,43.725882299999995,-79.31557159999998,"43.725882299999995, -79.31557159999998"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359,"43.6542599, -79.3606359"
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718517999999996,-79.46476329999999,"43.718517999999996, -79.46476329999999"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623015,-79.3894938,"43.6623015, -79.3894938"


# Creating Graphs


In [12]:
address = 'Ontario, Canada'

geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Ontario are 50.000678, -86.000977.


In [13]:
# create map of New York using latitude and longitude values
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighbourhood']):
    label = '{}, {}'.format(merged_df, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

In [14]:
# Define foursquare credentials
CLIENT_ID = 'R2WUEKVJRJ3OGH4ZY01JMWXEIZ0TEZC01N2TFZXNMEK1Z3E0' # your Foursquare ID
CLIENT_SECRET = 'OZ4OFCIO5DJ0MNI0GRCKCW1CWJK1VCACALOVV0ES2NEECT5M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R2WUEKVJRJ3OGH4ZY01JMWXEIZ0TEZC01N2TFZXNMEK1Z3E0
CLIENT_SECRET:OZ4OFCIO5DJ0MNI0GRCKCW1CWJK1VCACALOVV0ES2NEECT5M


In [15]:
merged_df.loc[0, 'Neighbourhood']

'Parkwoods'

In [16]:
neighborhood_latitude = merged_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = merged_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = merged_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [17]:
#Getting the top 100 venues that are in Parkwoods within 500 meters
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=R2WUEKVJRJ3OGH4ZY01JMWXEIZ0TEZC01N2TFZXNMEK1Z3E0&client_secret=OZ4OFCIO5DJ0MNI0GRCKCW1CWJK1VCACALOVV0ES2NEECT5M&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [18]:
results= requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '603b0b741a62722185af2589'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
#clean the JSON and structure it into a pandas DF
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


# Explore all the neighborhoods in Ontario

In [22]:
#Function to do the same process for all neighborhoods in Ontario
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
#calling the above function to create a new DF called ontario_venues
ontario_venues = getNearbyVenues(names=merged_df['Neighbourhood'],
                                   latitudes=merged_df['Latitude'],
                                   longitudes=merged_df['Longitude']
                                  )
print("***Done Printing Neighborhoods! ****")

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [24]:
#Check the data
print(ontario_venues.shape)
ontario_venues.head()

(2114, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7532586,-79.3296565,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7532586,-79.3296565,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882299999995,-79.31557159999998,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882299999995,-79.31557159999998,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882299999995,-79.31557159999998,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [25]:
#How many venues are in each neighborhood
ontario_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
"Willowdale, Willowdale East",34,34,34,34,34,34
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,4,4,4,4,4,4


In [26]:
# How many unique venues there are
print('There are {} uniques categories.'.format(len(ontario_venues['Venue Category'].unique())))

There are 268 uniques categories.


# Analyze individual neighborhoods


In [27]:
# one hot encoding
ontario_onehot = pd.get_dummies(ontario_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ontario_onehot['Neighborhood'] = ontario_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ontario_onehot.columns[-1]] + list(ontario_onehot.columns[:-1])
ontario_onehot = ontario_onehot[fixed_columns]

ontario_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
ontario_onehot.shape

(2114, 268)

In [29]:
#Group the rows by neighborhood and by taking the mean of the frequency of occurence of each category
ontario_grouped = ontario_onehot.groupby('Neighborhood').mean().reset_index()
ontario_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.00
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00


In [30]:
ontario_grouped.shape

(96, 268)

In [31]:
#printing each neighborhood with the top 5 common venues
num_top_venues = 5

for hood in ontario_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ontario_grouped[ontario_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3             Clothing Store  0.25
4         Mexican Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1        Pharmacy  0.14
2  Sandwich Place  0.14
3     Coffee Shop  0.14
4             Pub  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0                 Bank  0.11
1          Coffee Shop  0.11
2          Supermarket  0.05
3     Sushi Restaurant  0.05
4  Fried Chicken Joint  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4  Moroccan Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1      Sandwich Place  0.08
2  Italian Restaurant  0.08
3    Gr

                  venue  freq
0           Pizza Place  0.18
1             Gastropub  0.09
2              Pharmacy  0.09
3  Gym / Fitness Center  0.09
4        Breakfast Spot  0.09


----Parkwoods----
                      venue  freq
0                      Park   0.5
1         Food & Drink Shop   0.5
2               Yoga Studio   0.0
3  Mediterranean Restaurant   0.0
4        Miscellaneous Shop   0.0


----Queen's Park, Ontario Provincial Government----
                venue  freq
0         Coffee Shop  0.20
1    Sushi Restaurant  0.07
2         Yoga Studio  0.03
3  Mexican Restaurant  0.03
4                 Bar  0.03


----Regent Park, Harbourfront----
            venue  freq
0     Coffee Shop  0.18
1            Park  0.07
2          Bakery  0.07
3  Breakfast Spot  0.04
4            Café  0.04


----Richmond, Adelaide, King----
             venue  freq
0      Coffee Shop  0.11
1             Café  0.05
2       Restaurant  0.04
3   Clothing Store  0.03
4  Thai Restaurant  0.03


----Ros

# Putting this data into a Pandas DF

In [32]:
#sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
# Create DF that shows top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ontario_grouped['Neighborhood']

for ind in np.arange(ontario_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ontario_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Clothing Store,Breakfast Spot,Eastern European Restaurant,Dumpling Restaurant,Electronics Store,Drugstore,Department Store,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Pub,Sandwich Place,Diner,Deli / Bodega,Department Store,Dessert Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Shopping Mall,Fried Chicken Joint,Sushi Restaurant,Ice Cream Shop,Restaurant,Middle Eastern Restaurant,Deli / Bodega,Mobile Phone Shop
3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Pub,Pizza Place,Pharmacy,Liquor Store,Juice Bar,Japanese Restaurant


In [34]:
neighborhoods_venues_sorted.columns

Index(['Neighborhood', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

# K means Clustering 

In [35]:
# K Means clustering to cluster the neighborhood into 5 groups
# set number of clusters
kclusters = 5

ontario_grouped_clustering = ontario_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ontario_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [36]:
ontario_grouped_clustering.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04


In [40]:
# Creating new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ontario_merged = merged_df

# merge ontario_grouped with ontario_data to add latitude/longitude for each neighborhood
ontario_merged = ontario_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

ontario_merged.head() # check the last columns!




,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7532586,-79.3296565,"43.7532586, -79.3296565",0.0,Park,Food & Drink Shop,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.725882299999995,-79.31557159999998,"43.725882299999995, -79.31557159999998",1.0,Portuguese Restaurant,Coffee Shop,Intersection,Pizza Place,French Restaurant,Hockey Arena,Financial or Legal Service,Diner,Department Store,Dessert Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359,"43.6542599, -79.3606359",1.0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Pub,Theater,Beer Store,Hotel,Dessert Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718517999999996,-79.46476329999999,"43.718517999999996, -79.46476329999999",1.0,Clothing Store,Shoe Store,Coffee Shop,Furniture / Home Store,Boutique,Miscellaneous Shop,Vietnamese Restaurant,Event Space,Accessories Store,Dumpling Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623015,-79.3894938,"43.6623015, -79.3894938",1.0,Coffee Shop,Sushi Restaurant,Gym,Fried Chicken Joint,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place


In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ontario_merged['Latitude'], ontario_merged['Longitude'], ontario_merged['Neighbourhood'], ontario_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

In [53]:
#Cluster 1-- These have Park as the most common Venue
examine= ontario_merged.loc[ontario_merged['Cluster Labels'] == 0, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]
print("For cluster {}, the distribution of venues is as:\n{}".format(0, examine['1st Most Common Venue'].value_counts()))
examine.head()

For cluster 0, the distribution of venues is as:
Park    10
Name: 1st Most Common Venue, dtype: int64


,Borough,Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,"43.7532586, -79.3296565",0.0,Park,Food & Drink Shop,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Ethiopian Restaurant
21,York,"43.6890256, -79.453512",0.0,Park,Pool,Women's Store,College Rec Center,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
35,East York,"43.685347, -79.3381065",0.0,Park,Convenience Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio
52,North York,"43.789053, -79.40849279999999",0.0,Park,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
64,York,"43.706876, -79.51818840000001",0.0,Park,Jewelry Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


In [54]:
#Cluster 2-- These have Coffee shop & Park as the most common
examine= ontario_merged.loc[ontario_merged['Cluster Labels'] == 1, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]
print("For cluster {}, the distribution of venues is as:\n{}".format(1, examine['1st Most Common Venue'].value_counts()))
examine.head()

For cluster 1, the distribution of venues is as:
Coffee Shop                  19
Park                         12
Pizza Place                   6
Café                          5
Gym                           3
Grocery Store                 3
Sandwich Place                3
Fast Food Restaurant          2
Sporting Goods Shop           2
Trail                         2
Clothing Store                2
Bakery                        2
Light Rail Station            1
Chinese Restaurant            1
Gift Shop                     1
Motel                         1
Golf Course                   1
Skating Rink                  1
Department Store              1
Latin American Restaurant     1
College Stadium               1
Garden Center                 1
Indian Restaurant             1
Restaurant                    1
Portuguese Restaurant         1
Baseball Field                1
Fabric Shop                   1
Bus Line                      1
Massage Studio                1
Auto Garage            

,Borough,Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,"43.725882299999995, -79.31557159999998",1.0,Portuguese Restaurant,Coffee Shop,Intersection,Pizza Place,French Restaurant,Hockey Arena,Financial or Legal Service,Diner,Department Store,Dessert Shop
2,Downtown Toronto,"43.6542599, -79.3606359",1.0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Pub,Theater,Beer Store,Hotel,Dessert Shop
3,North York,"43.718517999999996, -79.46476329999999",1.0,Clothing Store,Shoe Store,Coffee Shop,Furniture / Home Store,Boutique,Miscellaneous Shop,Vietnamese Restaurant,Event Space,Accessories Store,Dumpling Restaurant
4,Downtown Toronto,"43.6623015, -79.3894938",1.0,Coffee Shop,Sushi Restaurant,Gym,Fried Chicken Joint,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place
5,Etobicoke,"43.6678556, -79.53224240000002",1.0,Pizza Place,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


In [55]:
#Cluster 3-- This has 1 park 
examine= ontario_merged.loc[ontario_merged['Cluster Labels'] == 2, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]
print("For cluster {}, the distribution of venues is as:\n{}".format(1, examine['1st Most Common Venue'].value_counts()))
examine.head()

For cluster 1, the distribution of venues is as:
Playground    1
Name: 1st Most Common Venue, dtype: int64


,Borough,Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,"43.7447342, -79.23947609999999",2.0,Playground,Cosmetics Shop,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


In [56]:
#Cluster 4-- This has 1 bar
examine= ontario_merged.loc[ontario_merged['Cluster Labels'] == 3, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]
print("For cluster {}, the distribution of venues is as:\n{}".format(1, examine['1st Most Common Venue'].value_counts()))
examine.head()

For cluster 1, the distribution of venues is as:
Bar    1
Name: 1st Most Common Venue, dtype: int64


,Borough,Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,"43.7845351, -79.16049709999999",3.0,Bar,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Event Space


In [57]:
#Cluster 5-- This 2 fast food restaurants
examine= ontario_merged.loc[ontario_merged['Cluster Labels'] == 4, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]
print("For cluster {}, the distribution of venues is as:\n{}".format(1, examine['1st Most Common Venue'].value_counts()))
examine.head()

For cluster 1, the distribution of venues is as:
Fast Food Restaurant    2
Name: 1st Most Common Venue, dtype: int64


,Borough,Coordinates,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,"43.806686299999996, -79.19435340000001",4.0,Fast Food Restaurant,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
62,Central Toronto,"43.7116948, -79.41693559999999",4.0,Fast Food Restaurant,Garden,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
